In [1]:
import cvxpy as cp
import numpy as np

## Nominal optimization problem

In [15]:
cost1 = 700
cost2 = 800
A = np.array([[-0.01, -0.02, 0.5, 0.6],
                  [1, 1, 0, 0],
                  [0, 0, 90.0, 100.0],
                  [0, 0, 40.0, 50.0],
                  [100.0, 199.9, cost1, cost2]])
b = np.array([0, 1000, 2000, 800, 100000])
c = -np.array([100, 199.9,  cost1 - 6200,  cost2 - 6900])

x = cp.Variable(4)

obj = cp.Maximize(c.T @ x)

constraints = [x >= 0,
               A@x <= b]

prob = cp.Problem(obj, constraints)
prob.solve()
robustX = x.value
robustProfit = prob.value
print('robustX: ', np.round(robustX, 3))
print('robustProfit: ', robustProfit)
print('')

robustX:  [  0.    438.789  17.552   0.   ]
robustProfit:  8819.657742458627



The optimal solution prescribes to purchase 438.8 kg of RawII and to produce 17.552K packs of DrugI, or in otherwords, don't buy RawI and don't produce DrugII. 

Consider the case in which variability in the drug content of RawI exists within the bounds $a_1 = [0.00995, 0.01005]$, a $0.5\%$ drift, and Raw2 exists within the bounds $a_2 = [0.0196, 0.0204]$, or a $2.0\%$ drift. Since the above plan prescribes only purchasing RawII, the worst case scenario implies that the active agent content of RawII contains $2\%$ less than expected, resulting in $2\%$ less DrugI produced, 17.201K packs produced. This means that total profit reduces down to

$$-\$199.9/kg \cdot 438.789kg - \left(1 - 0.02\right) \cdot 17.552k \textit{ pack} \cdot \$ 6200 / pack + \left(1 - 0.02\right) \cdot 17.552k \textit{ pack} \cdot \$ 700 / pack = \$6,891.3589$$

This results in a profit that is $22\%$ less than the promised profit from the nominal optimization problem. 

## Robust Counterpart

Now maximize the problem given the box uncertainty (worst-case scenario). 

In [16]:
cost1 = 700
cost2 = 800
a1 = [0.01, 0.00995, 0.01005]        # with box uncertainty
a2 = [0.02, 0.0196, 0.0204]          # choose worst case scenario
A = np.array([[-(a1[1]), -(a2[1]), 0.5, 0.6],
                  [1, 1, 0, 0],
                  [0, 0, 90.0, 100.0],
                  [0, 0, 40.0, 50.0],
                  [100.0, 199.9, 700, 800]])
b = np.array([0, 1000, 2000, 800, 100000])
c = -np.array([100, 199.9,  cost1 - 6200,  cost2 - 6900])

x = cp.Variable(4)

obj = cp.Maximize(c.T @ x)

constraints = [x >= 0,
               A@x <= b]

prob = cp.Problem(obj, constraints)
prob.solve()
robustX = x.value
robustProfit = prob.value
print('robustX: ', np.round(robustX, 3))
print('robustProfit: ', robustProfit)
print('')

robustX:  [877.732   0.     17.467   0.   ]
robustProfit:  8294.566838742481



The robust counterpart indicates that it is better to purchase RawI and produce DrugI, with a 5.954\% lower possible profit than the nominal problem. This is much less than the 22\% reduction of the actual profit which may occur if the nominal plan is followed and the worst-case scenario occurs. 